In [3]:
import numpy as np
import yfinance as yf
from scipy.stats import norm
import pandas as pd
import datetime

In [4]:
def download_data(stock, start_date, end_date):
    data = {}
    ticker = yf.download(stock, start_date, end_date)
    data[stock] = ticker["Adj Close"]
    return pd.DataFrame(data)

In [9]:
def calculate_var(position, c, mu, sigma):
    # VaR tomorrow, ie (n=1)
    var = position * (mu - sigma*norm.ppf(1-c))
    return var

In [19]:
def calculate_var_n(position, c, mu, sigma, n):
    # VaR n days in the future, ie (n>1)
    var = position * (mu*n - sigma*np.sqrt(n)*norm.ppf(1-c))
    return var

In [28]:
class ValueAtRiskMonteCarlo:
    def __init__(self, S, mu, sigma, c, n, iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations
    
    def simulation(self):
        rand = np.random.normal(0, 1, [1, self.iterations])

        stock_price = self.S * np.exp(self.n * (self.mu - 0.5 * self.sigma**2) + self.sigma * np.sqrt(self.n) * rand)

        # sort to determine percentile
        stock_price = np.sort(stock_price)

        percentile = np.percentile(stock_price, (1-self.c)*100)

        return self.S - percentile

In [33]:
def main():
    start = datetime.datetime(2016, 1, 1)
    end = datetime.datetime(2021, 1, 1)
    stock_data = download_data("C", start, end)

    stock_data["returns"] = np.log(stock_data["C"] / stock_data["C"].shift(1))
    stock_data = stock_data[1:]
    

    # investment amt
    S = 1e6
    # CI
    c = 0.95
    # normality assmptn
    mu = np.mean(stock_data["returns"])
    sigma = np.std(stock_data["returns"])

    print("Value at risk (tomorrow) is: $%0.2f" % calculate_var(S, c, mu, sigma))
    print("Value at risk (in 10 days) is: $%0.2f" % calculate_var_n(S, c, mu, sigma, 10))

    model = ValueAtRiskMonteCarlo(S, mu, sigma, c, 10, 10000)
    print("Value at risk (in 10 days with MC) is : $%0.2f" % model.simulation())

In [34]:
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
Value at risk (tomorrow) is: $39330.07
Value at risk (in 10 days) is: $126005.48
Value at risk (in 10 days with MC) is : $115889.38
